In [1]:
#1. Import packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,callbacks,applications
import numpy as np
import os, datetime, pathlib
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
#2. Data preparation
file_path = r"/content/gdrive/MyDrive/roadcrack/road crack"
data_dir = pathlib.Path(file_path)

SEED = 12345
IMG_SIZE = (80,80)
BATCH_SIZE = 32

train_dataset = keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset='training',seed=SEED,shuffle=True,
                                                         image_size=IMG_SIZE,batch_size=BATCH_SIZE)
val_dataset = keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset='validation',seed=SEED,shuffle=True,
                                                         image_size=IMG_SIZE,batch_size=BATCH_SIZE)

Found 40000 files belonging to 2 classes.
Using 32000 files for training.
Found 40000 files belonging to 2 classes.
Using 8000 files for validation.


In [4]:
#Further split validation into validation-test splits
val_batches = tf.data.experimental.cardinality(val_dataset)
test_dataset = val_dataset.take(val_batches//5)
validation_dataset = val_dataset.skip(val_batches//5)


#Create prefetch dataset for all the 3 splits
AUTOTUNE = tf.data.AUTOTUNE
pf_train = train_dataset.prefetch(buffer_size=AUTOTUNE)
pf_val = validation_dataset.prefetch(buffer_size=AUTOTUNE)
pf_test = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [5]:
# Let's start to work on the model
# Our model will be:
# Data augmentation --> Preprocessing --> Feature extraction (pretrained model) --> Classification

#3. Data augmentation layers
data_augmentation = keras.Sequential()
data_augmentation.add(layers.RandomFlip('horizontal_and_vertical'))
data_augmentation.add(layers.RandomRotation(0.25))

In [6]:
#4. Model creation

class_names = train_dataset.class_names
nClass = len(class_names)
IMG_SHAPE = IMG_SIZE + (3,)
global_avg = layers.GlobalAveragePooling2D()

inputs = keras.Input(shape=IMG_SHAPE)
h1 = layers.Conv2D(32, (3,3), padding = "same", activation='relu')
h2 = layers.Conv2D(16, (3,3), padding = "same", activation='relu')
h3 = layers.Conv2D(8, (3,3), padding = "same", activation='relu')

c1 = layers.Flatten()
c3 = layers.Dense(32, activation='relu')
c4 = layers.Dense(16, activation='relu')
out_layer = layers.Dense(nClass, activation='softmax')

# Chain the functional API
x = data_augmentation(inputs)
x = x / 255.0 #(preprocess)
x = h1(x)
x = h2(x)
x = h3(x)
x = global_avg(x)
x = c1(x)
x = c3(x)
x = c4(x)
outputs = out_layer(x)

model = keras.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 80, 80, 3)         0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 80, 80, 3)        0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 80, 80, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 80, 80, 16)        4624      
                                                                 
 conv2d_2 (Conv2D)           (None, 80, 80, 8)         1160      
                                                             

In [7]:
#7. Compile the model
optimizer = optimizers.Adam(learning_rate = 0.001)
loss = losses.SparseCategoricalCrossentropy()

model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [8]:
#8. Perform model training
EPOCHS = 1
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 1272s 1s/step - loss: 0.4498 - accuracy: 0.7291 - val_loss: 0.0723 - val_accuracy: 0.9745


In [9]:
optimizer = optimizers.Adam(learning_rate = 0.0007)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 657s 656ms/step - loss: 0.0435 - accuracy: 0.9868 - val_loss: 0.0330 - val_accuracy: 0.9880


In [10]:
optimizer = optimizers.Adam(learning_rate = 0.00035)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 653s 652ms/step - loss: 0.0302 - accuracy: 0.9908 - val_loss: 0.0222 - val_accuracy: 0.9930


In [11]:
optimizer = optimizers.Adam(learning_rate = 0.00022)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 649s 649ms/step - loss: 0.0270 - accuracy: 0.9913 - val_loss: 0.0199 - val_accuracy: 0.9937


In [12]:
optimizer = optimizers.Adam(learning_rate = 0.00012)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 654s 654ms/step - loss: 0.0246 - accuracy: 0.9918 - val_loss: 0.0184 - val_accuracy: 0.9934


In [13]:
optimizer = optimizers.Adam(learning_rate = 0.00008)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 666s 665ms/step - loss: 0.0208 - accuracy: 0.9931 - val_loss: 0.0149 - val_accuracy: 0.9958


In [14]:
optimizer = optimizers.Adam(learning_rate = 0.000045)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 652s 651ms/step - loss: 0.0196 - accuracy: 0.9942 - val_loss: 0.0164 - val_accuracy: 0.9950


In [15]:
optimizer = optimizers.Adam(learning_rate = 0.00003)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 619s 618ms/step - loss: 0.0186 - accuracy: 0.9941 - val_loss: 0.0216 - val_accuracy: 0.9928


In [19]:
optimizer = optimizers.Adam(learning_rate = 0.00009)
history = model.fit(pf_train,validation_data=pf_val,epochs=EPOCHS)

1000/1000 [==============================] - 617s 616ms/step - loss: 0.0188 - accuracy: 0.9942 - val_loss: 0.0146 - val_accuracy: 0.9959


In [20]:
#Evaluate the model after training
test_loss, test_accuracy = model.evaluate(pf_test)

print("------------------------------After Training---------------------------")
print("Loss = ", test_loss)
print("Accuracy = ",test_accuracy)

50/50 [==============================] - 8s 156ms/step - loss: 0.0139 - accuracy: 0.9969
------------------------------After Training---------------------------
Loss =  0.013866939581930637
Accuracy =  0.996874988079071


In [21]:
#Deploy the model to make predictions
image_batch, label_batch = pf_test.as_numpy_iterator().next()
predictions = np.argmax(model.predict(image_batch),axis=1)

In [22]:
#Compare label vs prediction
label_vs_prediction = np.transpose(np.vstack((label_batch,predictions)))

In [23]:
print(label_vs_prediction)

[[0 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]]
